zebra puzzles

In [13]:
from copy import deepcopy
from random import randint
from icecream import ic
from tqdm.auto import tqdm
from random import choice, random, sample, randrange

In [14]:
NUM_ITEM = 50
NUM_CATEGORIES = 10


In [15]:
ITEMS = [[(c,n) for n in range(NUM_ITEM)] for c in range(NUM_CATEGORIES)]
INCOMPATIBLE = [{randint(0,NUM_ITEM-1),randint(0,NUM_ITEM-1) , (randint(0,NUM_CATEGORIES-1),randint(0,NUM_CATEGORIES-1))} for _ in range(10000)]


In [16]:
def check(bag: set) -> bool:
    return any(i <= bag for i in INCOMPATIBLE)
# def cost(solution: list[set]) -> int:
#     return sum(check(b) for b in solution)

#for debugging
def cost(solution):
    # 代价 = 违规袋子的数量（越小越好；0 表示全可行）
    return sum(1 for bag in solution if not check(bag))


In [ ]:


#1.all mutation is possible
#2.mutation is not too big -- small change
#3.expolration vs exploitation -- different phase of the search
# def tweak(bag: list[set],t:float = 0.1) -> list[set]:
#     new_bags = deepcopy(bag)

#     again = True
#     while again:
#         # b1 = choice(i for i in range(NUM_ITEM) if check(new_bags[i]))
#         candidates = [i for i in range(NUM_ITEM) if invalid(new_bags[i])]
#         if not candidates:
#             # 没有任何满足 check 的袋子，直接返回或打破循环
#             return new_bags  # 或者：break

#         b1 = choice(candidates)
#         b2 = randint(0,NUM_ITEM-1)
#         b1_,b2_ = sorted(new_bags[b1]),sorted(new_bags[b2])
#         i = randint(0,NUM_CATEGORIES-1)
#         b1_[i],b2_[i] = b2_[i],b1_[i]
#         new_bags[b1],new_bags[b2] = set(b1_),set(b2_)
#         again = random() < t
#     return new_bags


#new tweak for debugging
def tweak(current_solution, t=0.2):
    # 逐袋 copy，避免浅拷贝
    new_bags = [b.copy() for b in current_solution]

    # 先构造列表；为空要兜底，否则 random.choice 会报错
    candidates = [i for i in range(NUM_ITEM) if check(new_bags[i])]
    if not candidates:
        # 没有满足 check 的袋子，就从所有袋子随机挑两个不同的做扰动
        i, j = sample(range(NUM_ITEM), 2)
    else:
        i = choice(candidates)
        j = randrange(NUM_ITEM)
        if i == j:
            # 确保两个不同
            j = (j + 1) % NUM_ITEM

   
    # 从 i 袋拿一个放到 j 袋；如果本来是交换，就不变
    if new_bags[i]:
        itm = choice(list(new_bags[i]))
        new_bags[i].remove(itm)
        new_bags[j].add(itm)

    return new_bags

In [18]:
current_solution = [{ITEMS[c][i] for c in range(NUM_CATEGORIES)} for i in range(NUM_ITEM)]
current_cost = cost(current_solution)

ic(current_cost)

max_steps = 100

for steps in tqdm(range(1000)):
    if current_cost == 0:
        break
    new_solution = tweak(current_solution,t=0.2)
    new_cost = cost(new_solution)
    if new_cost < current_cost:
        current_solution = new_solution
        current_cost = new_cost 
        ic(current_cost)
print(current_cost)

ic| current_cost: 50
100%|██████████| 1000/1000 [00:29<00:00, 33.87it/s]

50
